# Decorator Factories

In [26]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args,**kwargs):
        
        start = perf_counter()
        result = fn(*args,**kwargs)
        end = perf_counter()
        elapsed = end-start
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1} '.format(k,v) for (k,v) in kwargs.items()]
        all_args_ = args_ + kwargs_
        args_str = ','.join(all_args_)
        print("{0}({1}) took {2:.6f} to run".format(fn.__name__, args_str,elapsed))
        return result
    return inner
    

In [31]:
def fib_recursive(n):
    if n < 2:
        return n
    return fib_recursive(n-1) + fib_recursive(n-2)

In [28]:
@timed
def fib(n):
    return fib_recursive(n)

In [37]:
fib(30)

Avg. run time : 0.151137 to run


832040

In [41]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args,**kwargs):
        total_elapsed = 0
        for i in range(10):
            start = perf_counter()
            result = fn(*args,**kwargs)
            end = perf_counter()
            total_elapsed += end-start
        
        avg_run_time = total_elapsed / 10
        print("Avg. run time : {0:.6f}s".format(avg_run_time))
        return result
    return inner
    

Here we are calculating average elapsed time by running the function 10 time and then calculating its avg.
Let's say we want to the avg for 20 calls or 25 calls then we need one more parameter.

In [43]:
@timed
def fib(n):
    return fib_recursive(n)

In [44]:
fib(35)

Avg. run time : 1.681875s


9227465

Trying to add parameter to decorator function

In [48]:
def timed(fn,raps):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args,**kwargs):
        total_elapsed = 0
        for i in range(raps):
            start = perf_counter()
            result = fn(*args,**kwargs)
            end = perf_counter()
            total_elapsed += end-start
        
        avg_run_time = total_elapsed / 10
        print("Avg. run time : {0:.6f}s for {1} raps ".format(avg_run_time,raps))
        return result
    return inner

In [49]:
def fib(n):
    return fib_recursive(n)

In [52]:
# Decotating the fucntion
fib = timed(fib,10)  # average is being calculated for 10 wraps

In [51]:
fib(10)

Avg. run time : 0.000014s for 10 raps 


55

But the problem over here is that we can't decorate a function using `@` operator like `timed(10)`

In [53]:
timed(10)
def fib(n):
    return fib_recursive(n)

TypeError: timed() missing 1 required positional argument: 'raps'

---
> To resolve this issue we need to create a decorator factory

In [62]:
def dec(fn):
    print("Running decorator")
    
    def inner(*args,**kwargs):
        print("running inner")
        return fn(*args,**kwargs)
    return inner

In [63]:
@dec
def my_func():
    print("Running my_func")
    

Running decorator


Decorating without @ operator

In [64]:
def my_func():
    print("Running my_func")

In [65]:
my_func = dec(my_func)

Running decorator


In [66]:
my_func()

running inner
Running my_func


Now creating a decorator factory, using which we can create multiple decorators

In [2]:
def dec_factory():
    print("Running decorator factory")
    def dec(fn):
        print("Running decorator")
        
        def inner(*args,**kwargs):
            print("running inner")
            return fn(*args,**kwargs)
        return inner
    return dec

In [3]:
# Create a decorator insgtance
dec = dec_factory()

Running decorator factory


In [4]:
@dec
def my_func():
    print("running my_func")

Running decorator


Now if we run `my_func` using decorator factory's new instance, it will act as above

In [5]:
my_func()

running inner
running my_func


#### Note
we know that -
- if we decleare a finction like this, `func` then it gives the function object but dosent call it.
- but if we decleare the function like this `func()` (_function name followed by `()`_) then it invokes(calls) 
the function
- so we can decorate a function like this - `@dec_factory()` this syntax will call the function which inturn returns a decorator, see the example below

In [6]:
@dec_factory()
def my_func():
    print("Running my func-2")

Running decorator factory
Running decorator


In [7]:
my_func()

running inner
Running my func-2


We know that we can also decorate a function like this

In [8]:
def my_func():
    print("Running my func-3")

In [11]:
# Decorating the function
my_func = dec_factory()(my_func)

# Above code corresponds to this
# dec = dec_factory()
# my_func = dec(my_func)


Running decorator factory
Running decorator


In [13]:
my_func()

running inner
running inner
Running my func-3


Now creating a decorator factory that takes parameter and these parameteres are passed to decorated functions

In [22]:
def dec_factory(a,b):
    print("Running decorator factory")
    def dec(fn):
        print("Running decorator")
        
        def inner(*args,**kwargs):
            print("running inner")
            print(f"values obtained from decorator factory -> a={a}, b={b}")
            return fn(*args,**kwargs)
        return inner
    return dec

In [23]:
dec = dec_factory(10,20)

Running decorator factory


In [36]:
@dec
def my_func():
    print("Runnig my_func")

Running decorator


In [25]:
my_func()

running inner
values obtained from decorator factory -> a=10, b=20
Runnig my_func


##### Other way to use this factory decorator is - 

In [26]:

@dec_factory(11,22)
def my_func():
    print("Runnig my_func")

Running decorator factory
Running decorator


In [27]:
my_func()

running inner
values obtained from decorator factory -> a=11, b=22
Runnig my_func


#### Now modefying `timed` decorator to take parameter `raps` to calculate average time  

In [28]:
def timed(raps):
    def dec(fn):
        from functools import wraps
        from time import perf_counter
        
        @wraps(fn)
        def inner(*args,**kwargs):
            total_elapsed = 0
            for i in range(raps):
                start = perf_counter()
                result = fn(*args,**kwargs)
                end = perf_counter()
                total_elapsed += end-start
            
            avg_run_time = total_elapsed / 10
            print("Avg. run time : {0:.6f}s for {1} raps ".format(avg_run_time,raps))
            return result
        return inner
    return dec

In [34]:
@timed(22)
def fib(n):
    return fib_recursive(n)

In [35]:
fib(5)

Avg. run time : 0.000002s for 22 raps 


5

> Now `@timed` is not a decorator anymore, insetd it is a decorator factory